In [1]:
!pip install langchain langchain_experimental

In [214]:
from operator import itemgetter

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain.callbacks.base import BaseCallbackHandler

from langchain_community.llms import Bedrock
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_experimental.chat_models import Llama2Chat
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_core.messages import SystemMessage

In [207]:
llm = Bedrock(
    model_id="meta.llama2-13b-chat-v1",
    model_kwargs={"temperature": 0.01},
    streaming=True,
)

template_messages = [
    SystemMessage(content="You are a smart assistant."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}"),
]

prompt = ChatPromptTemplate.from_messages(template_messages)

model = Llama2Chat(llm=llm, callbacks=[StreamingStdOutCallbackHandler()])

memory = ConversationBufferMemory(memory_key="history", return_messages=True)

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [216]:
inputs = {"input": "yes"}
response = chain.invoke(inputs)

memory.save_context(inputs, {"output": response.content})

TypeError: RunnableSequence.invoke() got an unexpected keyword argument 'callbacks'